In [1]:
import pandas_datareader as pdr
import requests
import pandas as pd
from datetime import datetime, timedelta
import sqlite3 
import sql_tools
import time
import numpy as np
from IPython.display import clear_output
pick = sql_tools.IcePick(sqlite3, 'trade_history.sqlite')
con = sqlite3.connect('trade_history.sqlite')


In [2]:
def download_data(ticker, *args, **kwargs):
    i = 0
    while True:
        try:
            print('Trying to Download: {}'.format(ticker))
            data = pdr.DataReader(ticker, 'yahoo', *args, **kwargs)
            return data
        except pdr.base.RemoteDataError as e:
            i = i + 1
            print(e)
            print('Error {} : Exponential Backoff time: {}'.format(i , i**2))
            
            if i > 7:
                print('Error Got too big, raising it')
                raise e 
            time.sleep(i ** 2)
            
def progress_bar(i, left):
        power_of_one_dot = 100/left
        pos = int(power_of_one_dot * i)
        bar = ''
        for num in range(0, pos -1 ):
            if num %2 == 0:
                bar = bar + 'o'
            if num %2 != 0:
                bar = bar + 'O'             
        bar = bar + '>'
        for num in range(0, 100 - pos):
            bar = bar + ' '
        bar = bar + '| {:.2f} % \nDone: {} Remaining: {}'.format(100 * i/left, i, left - i )
        print(bar)
        


In [69]:
def get_tickers():
    nyse = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download')
    nasdaq = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download')
    data = nyse.append(nasdaq)
    data['Symbol']= data['Symbol'].str.replace(' ', '')
    data = data.set_index('Symbol')
    data = data[~data.index.duplicated()]
    data = data[~data.index.str.contains('\^')]
    data = data[~data.index.str.contains('\.')]
    data = data[~data.index.str.contains('\~')]
    data = data[data['MarketCap'] > 0]
    del data['Unnamed: 9']
    return data
#tickers = get_tickers()

def create_updates(tickers):
    updates = pd.DataFrame(index = set(tickers))
    updates['last_updated'] = datetime(2018,1,24)
    updates.index.name = 'ticker'
    updates.to_sql('tickers', con, if_exists='replace')
    
def save_timestamp(ticker, time):
    cur = pick.get_connection()
    cur.execute("UPDATE tickers set last_updated = '{}' where ticker = '{}'".format(time,ticker))
    cur.commit()
    cur.close()
    return

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
def save_data(data, ticker = None, kind = ''):
    ''' kind must be one of:update, reset '''
    ok_types = ['update', 'reset']
    if kind not in ok_types:
        raise ValueError( ' argument kind must be one of the following: {}'.format(','.join(ok_types)))
    if (type(data) == pd.DataFrame) and (type(ticker) == type(None)):
        raise ValueError( 'Must Specify ticker argument with type = \'Dataframe\'')
    if type(data) == pd.core.panel.Panel:
        for ticker in data.minor_axis:
            candles = data.minor_xs(ticker)
            save_data(candles, ticker, kind)
            
    if type(data) == pd.DataFrame:
        data.columns = data.columns.str.replace(' ', '_')
        data = data.fillna("'nan'").replace('nan', "'nan'")
        if kind == 'reset':
            with pick.get_connection() as con:
                sql = 'DROP TABLE IF EXISTS {} '.format(ticker)
                con.execute(sql)
            with pick.get_connection() as con:    
                sql = """ create table {} (Date text primary key,
                Open real ,High real ,Low real ,
                Close real ,Adj_Close real, Volume real )""".format(ticker)
                con.execute(sql)
            kind = 'update'
            
        if kind == 'update':
            
            column_names = list([data.index.name]) + list(data.columns)
            sql = """ INSERT OR REPLACE INTO {}({})\nValues """.format(ticker,','.join(column_names) )
            value_bit = []
            for date in data.index:
                row = data.loc[date]
                others = ','.join(row.apply(str).values)
                value_bit.append( " ('{}', {} )".format(date, others))
            sql = sql + ',\n'.join(value_bit)
            with pick.get_connection() as con:
                con.execute(sql)
                con.commit()


        latest_date = data.index.max()
        save_timestamp(ticker, latest_date)
    return 
        

In [4]:
data = pick.get_data('tickers')
data = data.set_index('ticker')

In [ ]:
to_do = len(data)

for i,ticker in enumerate(data.index):
    try:
        current = pick.get_data(ticker).set_index("Date")
    except sqlite3.DatabaseError as e:
        current = download_data(ticker)
    save_data(current, ticker, 'reset')
    progress_bar(i, to_do)
    clear_output(wait = True)

oOoOoOoOoOoOoOoOoOoO>                                                                               | 21.54 % 
Done: 1167 Remaining: 4250


In [52]:
save_data(current, ticker, 'reset')

 INSERT OR REPLACE INTO CMPR(Date,Open,High,Low,Close,Adj_Close,Volume)
Values  ('2010-01-04 00:00:00', 56.889999,57.669998,55.93,56.25,56.25,1033000.0 ),
 ('2010-01-05 00:00:00', 56.700001,57.459999,56.509998,57.07,57.07,996600.0 ),
 ('2010-01-06 00:00:00', 57.470001,57.59,56.439999,57.060001,57.060001,388600.0 ),
 ('2010-01-07 00:00:00', 57.299999,57.459999,55.040001,55.290001,55.290001,634000.0 ),
 ('2010-01-08 00:00:00', 55.209999,56.200001,55.060001,55.43,55.43,345300.0 ),
 ('2010-01-11 00:00:00', 56.139999,56.470001,55.779999,55.98,55.98,809800.0 ),
 ('2010-01-12 00:00:00', 55.91,55.91,52.790001,53.860001,53.860001,791400.0 ),
 ('2010-01-13 00:00:00', 53.59,54.200001,52.669998,53.619999,53.619999,460000.0 ),
 ('2010-01-14 00:00:00', 53.68,54.290001,53.099998,54.0,54.0,381000.0 ),
 ('2010-01-15 00:00:00', 53.860001,54.369999,51.889999,52.0,52.0,620700.0 ),
 ('2010-01-19 00:00:00', 51.98,53.09,51.450001,52.900002,52.900002,572100.0 ),
 ('2010-01-20 00:00:00', 52.029999,53.119999,51

In [ ]:
def update_tickers(to_date):
    t = pd.to_datetime
    to_date = t(t(to_date).strftime('%Y-%m-%d'))
    tables = pick.get_data('tickers')
    tables = tables.set_index('ticker')
    tables['last_updated'] = pd.to_datetime(tables['last_updated'])

    tables['need_update'] = tables['last_updated'] < to_date
    tables=  tables[tables['need_update']]
    
    i = 0
    batch_size = 10
    to_do_length = len(tables)
    for thing in batch(tables, batch_size):
        downloaded_data = download_data(thing.index)
        downloaded_data = pd.DataFrame()
        save_data(downloaded_data, kind = 'update')
        
        i = min(i + batch_size, to_do_length)
        progress_bar(i,to_do_length)
        clear_output(True)
        
    return tables
#x = update_tickers(datetime.today())